In [14]:
from elevatorSystem import *
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.optimizers import Adam
from rl.agents import DQNAgent
from rl.policy import BoltzmannGumbelQPolicy
from rl.memory import SequentialMemory~
from stable_baselines3 import PPO

In [2]:
n_elevators = 3
system = ElevatorSystem(n_elevators,20)
plays = system.actions()
for episode in range(1,2):
    system.reset()
    done = False
    score = 0    
    while not done:        
        obs, reward, done, info = system.step(random.choice(plays))
        score += reward

    print('Episode:{} Score: {}'.format(episode,score))

Episode:1 Score: -2824


In [3]:
info

{'Elevators': [{'Current Floor': 0,
   'Doors Open': True,
   'Max Capacity': 4,
   'Current Passengers': 4,
   'Destinations': array([ 6,  9,  6, 11])},
  {'Current Floor': 3,
   'Doors Open': False,
   'Max Capacity': 4,
   'Current Passengers': 4,
   'Destinations': array([14, 14, 16, 15])},
  {'Current Floor': 20,
   'Doors Open': True,
   'Max Capacity': 4,
   'Current Passengers': 4,
   'Destinations': array([ 7, 15, 14,  3])}],
 'Floors waiting': [0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20]}

In [4]:
obs

array([ 0.,  1.,  4.,  4.,  6.,  9.,  6., 11.,  3.,  0.,  4.,  4., 14.,
       14., 16., 15., 20.,  1.,  4.,  4.,  7., 15., 14.,  3.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.])

In [5]:
obs.shape[0]

44

In [7]:
def build_model(states, actions):
    model = Sequential()
    model.add(Flatten(input_shape=(1,states)))
    model.add(Dense(24,activation='relu'))
    model.add(Dense(24,activation='relu'))
    model.add(Dense(actions,activation='linear'))
    return model

In [12]:
model = build_model(obs.shape[0],plays.shape[0])
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_2 (Flatten)         (None, 44)                0         
                                                                 
 dense_6 (Dense)             (None, 24)                1080      
                                                                 
 dense_7 (Dense)             (None, 24)                600       
                                                                 
 dense_8 (Dense)             (None, 10)                250       
                                                                 
Total params: 1,930
Trainable params: 1,930
Non-trainable params: 0
_________________________________________________________________


In [17]:
def build_agent(model):
    policy = BoltzmannGumbelQPolicy()
    dqn = DQNAgent(model=model,policy=policy)
    return dqn

In [18]:
dqn = build_agent(model)
dqn.compile(Adam(lr=1e-3),metrics=['mae'])
dqn.fit(system, nb_steps=50000, verbose=1)

TypeError: AbstractDQNAgent.__init__() missing 2 required positional arguments: 'nb_actions' and 'memory'